In [1]:
##Visualizations for exploration & presentation

In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import mapclassify
import numpy as np
import seaborn as sns
import wordcloud
from wordcloud import WordCloud, STOPWORDS 
import copy

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
#allow in line graphics to view the plots
%matplotlib inline

In [ ]:
#resize the graphics - will likely update 
plt.rcParams['figure.figsize'] = (20, 10)

In [ ]:
#geojson file from the city of chicago data portal to graph the neighborhood boundaries
#might need to update the path
chicago_neighborhoods = gpd.read_file('Chicago_Data/chicago_boundaries.geojson')

In [ ]:
chicago_neighborhoods.head()

In [ ]:
#plot just the neighborhood boundaries? 
chicago_neighborhoods.boundary.plot(color = "black");

In [ ]:
#need to create a layer of the restaurants 
#load in the restaurants data frame which should include lat/long - thanks for the update Egemen
restaurant = pd.read_csv("CSVs/all_businesses.csv", index_col =0 )
restaurant.head()

In [ ]:
#turn this into a shapely object for the layer 
rest_shapely = gpd.GeoDataFrame(
    restaurant, geometry=gpd.points_from_xy(restaurant.longitude, restaurant.latitude))

In [ ]:
#All businesses weirdly includes locations outside of Chicago which were ruining maps, subset to only include Chicago 
rest_chicago = rest_shapely[rest_shapely.city == "Chicago"]

In [ ]:
rest_closed = rest_chicago[rest_chicago.is_closed == True]
rest_open = rest_chicago[rest_chicago.is_closed == False]

In [ ]:
#Layer these  
fig,ax = plt.subplots(figsize = (20,20))
ax.set_facecolor('silver')
chicago_neighborhoods.plot(ax=ax, color = "white", edgecolor = "black")
rest_closed.plot(ax=ax, color = "red", alpha = 0.4, label = "Closed")
rest_open.sample(n=1000).plot(ax=ax, color = "blue", alpha = 0.2, label = "Open")
plt.legend(prop = {"size":15})
#plt.savefig('restaurants.png')

In [ ]:
#some work in Jade's Geoplot to plot COVID data as bubble plots - not very successful 
#won't include here 

In [ ]:
##Egemen's Code

In [ ]:
#identify the number of closed businesses per zipcode
all_businesses = pd.read_csv('CSVs/all_businesses.csv', index_col=0, dtype={'zip_code': str})
closed = all_businesses[all_businesses.is_closed]
zipcodes_closed = closed.loc[:, ['id', 'zip_code']].groupby('zip_code').count().sort_values('id', ascending=False)
zipcodes_closed

In [ ]:
#plot of the top 10 zipcodes with the most closed businesses (most affected)
closed_by_zip_top10 = sns.barplot(x=zipcodes_closed.index[:10], y=zipcodes_closed.id[:10], palette="Blues_d", order=zipcodes_closed.index[:10])
fig = closed_by_zip_top10.get_figure()
#fig.savefig('top10closedzips.png')

In [ ]:
#plot of the bottom 10 zipcodes with the most closed businesses (least affected)
closed_by_zip_bottom10 = sns.barplot(x=zipcodes_closed.index[-12:-2], y=zipcodes_closed.id[-12:-2], palette="Greens_d", order=zipcodes_closed.index[-12:-2])
fig = closed_by_zip_top10.get_figure()
#fig.savefig('bottom10closedzips.png')

In [ ]:
_open = all_businesses[all_businesses.is_closed==False]
zipcodes_open = _open.loc[:, ['id', 'zip_code']].groupby('zip_code').count().sort_values('id', ascending=False)

In [ ]:
#ratio of closed to open 
zip_ratio_closed = zipcodes_open.merge(zipcodes_closed, left_index=True, right_index=True, how='right')
zip_ratio_closed['ratio_closed'] = zip_ratio_closed.id_y/zip_ratio_closed.id_x
zip_ratio_closed = zip_ratio_closed.iloc[:, [-1]]
zip_ratio_closed.sort_values('ratio_closed', ascending=False, inplace=True)

In [ ]:
#Geojson with Chicago zipcodes 
chicago_zipcodes = gpd.read_file('Chicago_Data/zipcodes1.geojson')


In [ ]:
chicago_zip_closed = chicago_zipcodes.merge(zip_ratio_closed, right_index=True, left_on='ZIP')

In [ ]:
#some zipcodes plotted to the same shape
zipcodes_closed.loc[['60601', '60602', '60603', '60604', '60605', '60611'], :].id.sum()

In [ ]:
zipcodes_closed.loc[['60606', '60607', '60661'], :].id.sum()

In [ ]:
zipcodes_closed.at['12311', ['id']] = zipcodes_closed.loc[['60601', '60602', '60603', '60604', '60605', '60611'], :].id.sum()

zipcodes_closed.at['6761', ['id']] = zipcodes_closed.loc[['60606', '60607', '60661'], :].id.sum()

zipcodes_closed.at['60610', ['id']] = zipcodes_closed.loc[['60610', '60654'], :].id.sum()

zipcodes_closed.at['60622', ['id']] = zipcodes_closed.loc[['60622', '60622'], :].id.sum()

zipcodes_open.at['12311', ['id']] = zipcodes_open.loc[['60601', '60602', '60603', '60604', '60605', '60611'], :].id.sum()

zipcodes_open.at['6761', ['id']] = zipcodes_open.loc[['60606', '60607', '60661'], :].id.sum()

zipcodes_open.at['60610', ['id']] = zipcodes_open.loc[['60610', '60654'], :].id.sum()

zipcodes_open.at['60622', ['id']] = zipcodes_open.loc[['60622', '60622'], :].id.sum()

zip_ratio_closed1 = zipcodes_open.merge(zipcodes_closed, left_index=True, right_index=True, how='right')
zip_ratio_closed1['ratio_closed'] = zip_ratio_closed1.id_y/zip_ratio_closed1.id_x
zip_ratio_closed1 = zip_ratio_closed1.iloc[:, [-1]]
zip_ratio_closed1.sort_values('ratio_closed', ascending=False, inplace=True)

chicago_zip_closed1 = chicago_zipcodes.merge(zip_ratio_closed1, right_index=True, left_on='ZIP')

In [ ]:
#use this to label neighborhoods
chicago_neighborhoods["center"] = chicago_neighborhoods["geometry"].centroid
za_points = chicago_neighborhoods.copy()
za_points.set_geometry("center", inplace = True)

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
ax.set_facecolor('white')
chicago_zip_closed1.plot(ax=ax, column=chicago_zip_closed1.ratio_closed, legend=True, cmap='Blues')
texts = []

for x, y, label in zip(za_points.geometry.x, za_points.geometry.y, za_points["community"]):
    texts.append(plt.text(x, y, label, fontsize = 8))
plt.savefig('ratio1.png')

In [ ]:
#look at COVID data
all_businesses['death_rate'] = all_businesses.covid_deaths / all_businesses.zipcode_population
all_businesses['case_rate'] = all_businesses.covid_cases / all_businesses.zipcode_population

rates = all_businesses.loc[:, ['zip_code', 'death_rate', 'case_rate']].drop_duplicates('zip_code')

chicago_rates_zip = chicago_zip_closed1.merge(rates, left_on='ZIP', right_on='zip_code', how='left')

In [ ]:
#pick representative COVID rates for the overlapping zipcodes
chicago_rates_zip.at[43, 'ZIP'] = 60601
chicago_rates_zip.at[43, 'death_rate'] = rates[rates.zip_code=='60601'].iat[0, 1]
chicago_rates_zip.at[43, 'case_rate'] = rates[rates.zip_code=='60601'].iat[0, 2]

chicago_rates_zip.at[44, 'ZIP'] = 60606
chicago_rates_zip.at[44, 'death_rate'] = rates[rates.zip_code=='60606'].iat[0, 1]
chicago_rates_zip.at[44, 'case_rate'] = rates[rates.zip_code=='60606'].iat[0, 2]

In [ ]:
#Deaths 
fig, ax = plt.subplots(figsize=(20,20))
ax.set_facecolor('white')
chicago_rates_zip.plot(ax=ax, column=chicago_rates_zip.death_rate, legend=True, cmap='OrRd')
texts = []

for x, y, label in zip(za_points.geometry.x, za_points.geometry.y, za_points["community"]):
    texts.append(plt.text(x, y, label, fontsize = 8))
plt.savefig('death_rates.png')

In [ ]:
#Cases
fig, ax = plt.subplots(figsize=(20,20))
ax.set_facecolor('white')
chicago_rates_zip.plot(ax=ax, column=chicago_rates_zip.case_rate, legend=True, cmap='RdPu')
texts = []

for x, y, label in zip(za_points.geometry.x, za_points.geometry.y, za_points["community"]):
    texts.append(plt.text(x, y, label, fontsize = 8))
plt.savefig('case_rates.png')

In [ ]:
##Sabina's visualizations
#for more detail, refer to Sabina_SummaryStats

In [ ]:
def create_word_cloud(dataframe, column, sep_val, stop_words, outputfilename):
    '''
    Function to create a wordcloud based on the text in a specified column of a specified
    dataframe. In the business dataset, the words in 'alias' are seperated by '-' but
    in name by ' ' so we specify that as a parameter as well. Additionally, if there are 
    stop words beyond those included in the wordcloud package, they can be added (adding 'chicago' 
    proves to clean many of our word cloud outputs)
    '''
    
    full_terms = ""
    for response in dataframe[column]:
        sep_resp = response.replace(sep_val, ' ').split(' ')
        for word in sep_resp: #append word
            full_terms += word.lower() + " " #" ".join(word.lower())+" "#.append(word.lower())

    mystopwords = copy.deepcopy(set(STOPWORDS))
    for stop_w in stop_words:
        mystopwords.add(stop_w)

    mywordcloud = WordCloud(width = 800, height = 800, 
                    background_color ='white', 
                    stopwords = mystopwords, 
                    min_font_size = 10).generate(full_terms) 

    # plot the WordCloud image                        
    plt.figure(figsize = (8, 8), facecolor = None) 
    plt.imshow(mywordcloud) 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 

    plt.show()
    
    mywordcloud.to_file(outputfilename)
    
    return full_terms
        
#alias_list = [alias_list.append(alias) for alias in all_businesses['alias']]

In [ ]:
means = {'type': ['review_count', 'rating'], 'closed': [round(closed[['review_count']].mean()[0],2), round(closed[['rating']].mean()[0], 2)], 'open': [round(_open[['review_count']].mean()[0], 2), round(_open[['rating']].mean()[0],2)], 'all_businesses': [round(all_businesses[['review_count']].mean()[0], 2), round(all_businesses[['rating']].mean()[0], 2)]}
pd.DataFrame(means)

In [ ]:
busi_type = copy.deepcopy(all_businesses.groupby("business_type"))
#to avoid changing the original dataset
desc_df = pd.DataFrame(busi_type.describe())
#.sort_values('count', ascending=False)
desc_df['review_count'].sort_values('count', ascending=False)[0:20]

In [ ]:
## Word density plot of the alias (descriptors) for ALL businesses in chicago

In [ ]:
all_businesses = restaurant

cleaned_alias = create_word_cloud(all_businesses, 'alias', '-', ['chicago'], "all_alias_wordcloud.png")

uncleaned_res_name = create_word_cloud(all_businesses, 'name', ' ', [], "all_name_wordcloud.png")

In [ ]:
## Word clouds of the alias (descriptors) and restaurant names for open and closed businesses in chicago - as a comparison

In [ ]:
closed_alias_wc = create_word_cloud(closed, 'alias', '-', ['chicago'], "closed_alias_wordcloud.png")

open_alias_wc = create_word_cloud(_open, 'alias', '-', ['chicago'], "open_alias_wordcloud.png")

closed_name_wc = create_word_cloud(closed, 'name', '-', [], "closed_name_wordcloud.png")

open_name_wc = create_word_cloud(_open, 'name', '-', [], "open_name_wordcloud.png")

In [ ]:
rating_zipc_df = _open.groupby(['zip_code'],as_index = True)['rating'].mean()
reviews_zipc_df = _open.groupby(['zip_code'],as_index = True)['review_count'].mean()
merge_rating_reviews = pd.merge(rating_zipc_df, reviews_zipc_df, on='zip_code')

In [ ]:
closed_rating_zipc_df = closed.groupby(['zip_code'],as_index = True)['rating'].mean()
closed_reviews_zipc_df = closed.groupby(['zip_code'],as_index = True)['review_count'].mean()

closed_merge_rating_reviews = pd.merge(closed_rating_zipc_df, closed_reviews_zipc_df, on='zip_code')

closed_rate_mean = closed_merge_rating_reviews['rating'].mean()
closed_merge_rating_reviews["rating_eval"] = ["above average" if rate > closed_rate_mean else "below average" for rate in closed_merge_rating_reviews['rating']]
closed_merge_rating_reviews.reset_index(inplace=True)

merge_rating_reviews['is_closed'] = False
closed_merge_rating_reviews['is_closed'] = True
all_merge_rating_reviews = pd.concat([merge_rating_reviews, closed_merge_rating_reviews])

all_merge_rating_reviews


In [ ]:
#ax = sns.barplot(x="day", y="total_bill", hue="sex", data=tips)
sns.set_context("paper", font_scale=1.2)
sns.set_theme(style="whitegrid")
avg_rating_eval = sns.barplot(x = "zip_code", y = "review_count", hue = "is_closed", data = all_merge_rating_reviews).set_title("Average Review Counts per Zip Code of both Open and Closed Businesses")
fig = avg_rating_eval.get_figure()
fig.set_size_inches(15, 8.27)
fig.savefig('ALL_avg_reviews.png')


In [ ]:
sns.set_context("paper", font_scale=1.2)
sns.set_theme(style="whitegrid")
avg_rating_eval = sns.barplot(x = "zip_code", y = "rating", hue = "is_closed", data = all_merge_rating_reviews).set_title("Average Rating per Zip Code for both Open and Closed Businesses")
fig = avg_rating_eval.get_figure()
fig.set_size_inches(15, 8.27)
fig.savefig('ALL_avg_rating.png')

In [ ]:
merge_rating_reviews = pd.merge(rating_zipc_df, reviews_zipc_df, on='zip_code')
pd.DataFrame(merge_rating_reviews)

In [ ]:
rating_zipc_df.sort_values(ascending=False).head() #max = 5
reviews_zipc_df.sort_values(ascending=False).head() #max = 403
closed_rating_zipc_df.sort_values(ascending=False).head() #max = 5
closed_reviews_zipc_df.sort_values(ascending=False).head() #max = 310

In [ ]:
#ax = sns.barplot(x="day", y="total_bill", hue="sex", data=tips)

merge_rating_reviews.reset_index(inplace=True)
open_rate_mean = merge_rating_reviews['rating'].mean()
merge_rating_reviews["rating_eval"] = ["above average" if rate > open_rate_mean else "below average" for rate in merge_rating_reviews['rating']]
merge_rating_reviews.reset_index(inplace=True)

sns.set_context("paper", font_scale=1.2)
sns.set_theme(style="whitegrid")
avg_rating_eval = sns.barplot(x = "zip_code", y = "review_count", hue = "rating_eval", data = merge_rating_reviews).set_title("Scaled Mean Review Counts per Zip Code for Open Businesses")
fig = avg_rating_eval.get_figure()
fig.set_size_inches(15, 8.27)
fig.savefig('OPENavg_rating_eval.png')


In [ ]:
#ax = sns.barplot(x="day", y="total_bill", hue="sex", data=tips)
sns.set_context("paper", font_scale=1.2)
sns.color_palette("hls", 8)
closed_avg_rating_eval = sns.barplot(x = "zip_code", y = "review_count", hue = "rating_eval", data = closed_merge_rating_reviews).set_title("Scaled Mean Review Counts per Zip Code for Closed Businesses")
fig = closed_avg_rating_eval.get_figure()
fig.set_size_inches(15, 8.27)
fig.savefig('CLOSEDavg_rating_eval.png')



sns.set_context("paper", font_scale=1.2)
avg_rating_eval = sns.barplot(x = "zip_code", y = "review_count", hue = "rating_eval", data = merge_rating_reviews).set_title("Scaled Mean Review Counts per Zip Code")
fig = avg_rating_eval.get_figure()
fig.set_size_inches(15, 8.27)
fig.savefig('avg_rating_eval.png')
